## Install Required Libraries

In [1]:
# Cell 1: Install required packages
!pip install PyPDF2
!pip install pdfplumber
!pip install langchain-ibm
!pip install ibm-watsonx-ai
!pip install langchain
!pip install --upgrade langchain-community
!pip install faiss-cpu
!pip install smolagents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 89.5 MB/s eta 0:00:00
   ━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.7/81.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 8

## Standardized Pipeline Definition

We define a standardized pipeline for processing all documents, enforcing compliance with the required JSON structure. The pipeline remains consistent across all PDFs, while prompts are customized to extract data specific to each use case. To ensure accurate extraction and minimize error propagation in subsequent pipeline stages, we utilize Mistral Large for processing. The extracted data is then ingested and aggregated in later steps of the pipeline.

In [5]:
# Cell 2: Import required libraries
import PyPDF2
import pdfplumber
import os
import json
import logging
from typing import Optional, Dict, List
from pathlib import Path

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class PDFProcessor:
    """A class to handle PDF to text conversion using multiple backends."""

    # [Previous implementation remains the same]
    def __init__(self):
        """Initialize the PDF processor."""
        self.text_cache: Dict[str, str] = {}

    def extract_with_pdfplumber(self, file_path: str) -> Optional[str]:
        """Extract text using pdfplumber."""
        try:
            with pdfplumber.open(file_path) as pdf:
                pages = [page.extract_text() or '' for page in pdf.pages]
                return '\n'.join(pages)
        except Exception as e:
            logger.error(f"Error extracting text with pdfplumber: {str(e)}")
            return None

    def extract_with_pypdf2(self, file_path: str) -> Optional[str]:
        """Extract text using PyPDF2 as fallback."""
        try:
            with open(file_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                pages = [page.extract_text() or '' for page in reader.pages]
                return '\n'.join(pages)
        except Exception as e:
            logger.error(f"Error extracting text with PyPDF2: {str(e)}")
            return None

    def process_pdf(self, file_path: str) -> Optional[str]:
        """Process a single PDF file."""
        if not os.path.exists(file_path):
            logger.error(f"File not found: {file_path}")
            return None

        if file_path in self.text_cache:
            return self.text_cache[file_path]

        text = self.extract_with_pdfplumber(file_path) or self.extract_with_pypdf2(file_path)

        if text is None:
            logger.error(f"All extraction methods failed for {file_path}")
            return None

        self.text_cache[file_path] = text
        return text

In [6]:
# Cell 3: WatsonX LLM Configuration
from ibm_watsonx_ai import Credentials
from langchain_ibm import WatsonxLLM
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods

class WatsonXProcessor:
    """Class to handle WatsonX LLM processing."""

    def __init__(self):
        """Initialize WatsonX processor with credentials and parameters."""
        self.project_id = "44241d2e-6cff-49ce-b84c-1ed230f8eb36"
        self.api_key = "DZKT0-JkhDNn9_o6a_N3AiYk06HEifU3Kh6xnWXo16v-"
        self.credentials = Credentials(
            url="https://us-south.ml.cloud.ibm.com/",
            api_key=self.api_key
        )

        # Model configuration
        self.model_id = 'mistralai/mistral-large'
        self.parameters = {
            GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
            GenParams.MIN_NEW_TOKENS: 1,
            GenParams.MAX_NEW_TOKENS: 8192,
            GenParams.STOP_SEQUENCES: ["<|endoftext|>"],
            GenParams.TEMPERATURE: 0.9,
            GenParams.TOP_P: 0.9
        }

        # Initialize LLM
        self.llm = WatsonxLLM(
            model_id=self.model_id,
            url=self.credentials.get("url"),
            apikey=self.credentials.get("apikey"),
            project_id=self.project_id,
            params=self.parameters
        )

    def generate_response(self, prompt: str) -> str:
        """Generate response using WatsonX LLM."""
        formatted_prompt = f"<s>[INST] {prompt} [/INST]"
        try:
            response = self.llm.invoke(formatted_prompt)
            return response
        except Exception as e:
            logger.error(f"Error generating response: {str(e)}")
            return f"Error generating response: {str(e)}"

In [7]:
# Cell 4: Batch Processing Pipeline
class MenuExtractionPipeline:
    """Pipeline for processing multiple PDFs and extracting menu information."""

    def __init__(self, output_dir: str):
        """Initialize pipeline components."""
        self.pdf_processor = PDFProcessor()
        self.llm_processor = WatsonXProcessor()
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)

    def process_single_file(self, pdf_path: str, prompt: str) -> None:
        """Process a single PDF file and save its output."""
        logger.info(f"Processing {pdf_path}")

        # Extract text from PDF
        pdf_text = self.pdf_processor.process_pdf(pdf_path)
        if not pdf_text:
            logger.error(f"Failed to extract text from {pdf_path}")
            return

        # Generate full prompt
        full_prompt = f"""Dato il seguente testo:

        {pdf_text}

        {prompt}"""

        # Generate response
        response = self.llm_processor.generate_response(full_prompt)

        # Save output
        output_path = self.output_dir / f"{Path(pdf_path).stem}_menu.json"
        try:
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump({"text": response}, f, ensure_ascii=False, indent=2)
            logger.info(f"Saved output to {output_path}")
        except Exception as e:
            logger.error(f"Error saving output for {pdf_path}: {str(e)}")

    def process_directory(self, input_dir: str, prompt: str) -> None:
        """Process all PDF files in a directory."""
        input_path = Path(input_dir)
        if not input_path.exists():
            logger.error(f"Input directory {input_dir} does not exist")
            return

        pdf_files = list(input_path.glob("*.pdf"))
        logger.info(f"Found {len(pdf_files)} PDF files in {input_dir}")

        for pdf_file in pdf_files:
            self.process_single_file(str(pdf_file), prompt)

## Automated Extraction Pipelines

We define three distinct, partially and fully automated extraction pipelines to maintain high accuracy during the extraction phase:

1. **Metadata Extraction Pipeline**: Extracts metadata from documents, ensuring the foundational information is accurately captured.

2. **Ingredients and Techniques Extraction Pipeline**: Focuses on extracting ingredients and culinary techniques from menu documents.

3. **Constrained Extraction Pipeline**: Operates after processing all well-formatted, standardized, and uncorrupted documents. It informs the model of the known ingredients and techniques to guide and constrain the extraction process, achieving more effective and accurate results.

These pipelines work cohesively to optimize data extraction and ensure the integrity of the processed information.

In [9]:
# Cell 5: Prompt Definitions

EXTRACTION_CONFIGS = {
    "metadata": {
        "name": "Restaurant Metadata Extraction",
        "prompt": """Estrai da un PDF informazioni relative a ristoranti, restituendo un output in formato JSON con i seguenti dettagli per ogni ristorante:

- Nome del ristorante (dal titolo)
- Nome dello chef (sotto il titolo)
- Nome del pianeta (se presente)
- Licenze e skills in forma di elenco
- Tecnologie in forma di elenco

L'output deve avere questa struttura:

{
  "Nome_ristorante": "<nome_del_ristorante>",
  "Nome_chef": "<nome_dello_chef>",
  "Nome_pianeta": "<nome_del_pianeta>",
  "Licenze": [
    "<licenza 1>",
    "<licenza 2>",
    ...
  ],
  "Ordini": [
    "<ordine 1>",
    "<ordine 2>",
    ...
  ]
}""",
        "output_dir": "/content/Extracted_Metadata",
        "description": "Extracts restaurant metadata including name, chef, planet, and certifications"
    },

    "menu_basic": {
        "name": "Basic Menu Extraction",
        "prompt": """Estrai da un PDF un menu composto da più piatti, restituendo un output in formato JSON in cui ogni piatto è rappresentato con il nome, gli ingredienti e le tecniche utilizzate per la preparazione. L'output deve avere questa struttura:
{
  "<Nome Piatto>": {
    "Ingredienti": [
      "<Ingrediente1>",
      "<Ingrediente2>",
      ...
    ],
    "Tecniche": [
      "<Tecnica1>",
      "<Tecnica2>",
      ...
    ]
  }
}""",
        "output_dir": "/content/Extracted_Basic_Menu",
        "description": "Extracts menu items with ingredients and techniques"
    },

    "menu_constrained": {
        "name": "Constrained Menu Extraction",
        "prompt": """Estrai dal testo un menu composto da più piatti, restituendo un output in formato JSON in cui ogni piatto è rappresentato con il nome, gli ingredienti e le tecniche utilizzate per la preparazione. L'output deve avere questa struttura:
{
  "<Nome Piatto>": {
    "Ingredienti": [
      "<Ingrediente1>",
      "<Ingrediente2>",
      ...
    ],
    "Tecniche": [
      "<Tecnica1>",
      "<Tecnica2>",
      ...
    ]
  }
}
Assicurati di analizzare correttamente il testo per individuare i nomi dei piatti, le sezioni relative agli ingredienti e quelle relative alle tecniche.

Di seguito trovi l'elenco degli ingredienti esistenti:
{Affettamento a Pulsazioni Quantistiche, Affumicatura Polarizzata a Freddo Iperbarico, Affumicatura Psionica Sensoriale, Affumicatura Temporale Risonante, Affumicatura a Stratificazione Quantica, Affumicatura tramite Big Bang Microcosmico, Amalgamazione Sintetica Molecolare, Bollitura Entropica Sincronizzata, Bollitura Infrasonica Armonizzata, Bollitura Termografica a Rotazione Veloce, Congelamento Bio-Luminiscente Sincronico, Congelazione Iperdimensionalmente Stratificata, Cottura Idrodinamica Autoregolante, Cottura Olografica Quantum Fluttuante, Cottura Sottovuoto Antimateria, Cottura Sottovuoto Bioma Sintetico, Cottura Sottovuoto Frugale Energeticamente Negativa, Cottura Sottovuoto Multirealità Collassante, Cottura Sottovuoto Pulsar Magnetica, Cottura a Forno Dinamico Inversionale, Cottura a Vapore Ecodinamico Bilanciato, Cottura a Vapore Risonante Simbiotico, Cottura a Vapore Termocinetica Multipla, Cottura a Vapore con Flusso di Particelle Isoarmoniche, Cottura al Forno con Paradosso Temporale Cronospeculare, Cottura con Microonde Entropiche Sincronizzate, Cryo-Tessitura Energetica Polarizzata, Decostruzione Ancestrale, Decostruzione Atomica a Strati Energetici, Decostruzione Interdimensionale Lovecraftiana, Decostruzione Magnetica Risonante, Ebollizione Magneto-Cinetica Pulsante, Fermentazione Psionica Energetica, Fermentazione Quantica a Strati Multiversali, Fermentazione Quantico Biometrica, Fermentazione Temporale Sincronizzata, Grigliatura Eletro-Molecolare a Spaziatura Variabile, Grigliatura Plasma Sintetico Risonante, Grigliatura Psionica Dinamica Ritmica, Idro-Cristallizzazione Sonora Quantistica, Impasto Gravitazionale Vorticoso, Impasto a Campi Magnetici Dualistici, Incisione Elettromagnetica Plasmica, Marinatura Psionica, Marinatura Sotto Zero a Polarità Inversa, Marinatura Temporale Sincronizzata, Marinatura a Infusione Gravitazionale, Marinatura tramite Reazioni d'Antimateria Diluite, Modellatura Onirica Tetrazionale, Saltare in Padella Classica, Saltare in Padella Realtà Energetiche Parallele, Saltare in Padella Singolarità Inversa, Sferificazione Cromatica Interdimensionale, Sferificazione Filamentare a Molecole Vibrazionali, Sferificazione a Gravità Psionica Variabile, Sferificazione con Campi Magnetici Entropici, Sferificazione tramite Matrici Biofotiche, Sinergia Elettro-Osmotica Programmabile, Surgelamento Antimaterico a Risonanza Inversa, Taglio Dimensionale a Lame Fotofiliche, Taglio Sinaptico Biomimetico, Taglio a Risonanza Sonica Rigenerativa}

Di seguito trovi l'elenco delle tecniche esistenti:
{Affettamento a Pulsazioni Quantistiche, Affumicatura Polarizzata a Freddo Iperbarico, Affumicatura Psionica Sensoriale, Affumicatura Temporale Risonante, Affumicatura a Stratificazione Quantica, Affumicatura tramite Big Bang Microcosmico, Amalgamazione Sintetica Molecolare, Bollitura Entropica Sincronizzata, Bollitura Infrasonica Armonizzata, Bollitura Termografica a Rotazione Veloce, Congelamento Bio-Luminiscente Sincronico, Congelazione Iperdimensionalmente Stratificata, Cottura Idrodinamica Autoregolante, Cottura Olografica Quantum Fluttuante, Cottura Sottovuoto Antimateria, Cottura Sottovuoto Bioma Sintetico, Cottura Sottovuoto Frugale Energeticamente Negativa, Cottura Sottovuoto Multirealità Collassante, Cottura Sottovuoto Pulsar Magnetica, Cottura a Forno Dinamico Inversionale, Cottura a Vapore Ecodinamico Bilanciato, Cottura a Vapore Risonante Simbiotico, Cottura a Vapore Termocinetica Multipla, Cottura a Vapore con Flusso di Particelle Isoarmoniche, Cottura al Forno con Paradosso Temporale Cronospeculare, Cottura con Microonde Entropiche Sincronizzate, Cryo-Tessitura Energetica Polarizzata, Decostruzione Ancestrale, Decostruzione Atomica a Strati Energetici, Decostruzione Interdimensionale Lovecraftiana, Decostruzione Magnetica Risonante, Ebollizione Magneto-Cinetica Pulsante, Fermentazione Psionica Energetica, Fermentazione Quantica a Strati Multiversali, Fermentazione Quantico Biometrica, Fermentazione Temporale Sincronizzata, Grigliatura Eletro-Molecolare a Spaziatura Variabile, Grigliatura Plasma Sintetico Risonante, Grigliatura Psionica Dinamica Ritmica, Idro-Cristallizzazione Sonora Quantistica, Impasto Gravitazionale Vorticoso, Impasto a Campi Magnetici Dualistici, Incisione Elettromagnetica Plasmica, Marinatura Psionica, Marinatura Sotto Zero a Polarità Inversa, Marinatura Temporale Sincronizzata, Marinatura a Infusione Gravitazionale, Marinatura tramite Reazioni d'Antimateria Diluite, Modellatura Onirica Tetrazionale, Saltare in Padella Classica, Saltare in Padella Realtà Energetiche Parallele, Saltare in Padella Singolarità Inversa, Sferificazione Cromatica Interdimensionale, Sferificazione Filamentare a Molecole Vibrazionali, Sferificazione a Gravità Psionica Variabile, Sferificazione con Campi Magnetici Entropici, Sferificazione tramite Matrici Biofotiche, Sinergia Elettro-Osmotica Programmabile, Surgelamento Antimaterico a Risonanza Inversa, Taglio Dimensionale a Lame Fotofiliche, Taglio Sinaptico Biomimetico, Taglio a Risonanza Sonica Rigenerativa}

""",  # Full prompt omitted for brevity
        "output_dir": "/content/Extracted_Constrained_Menu",
        "description": "Extracts menu items with validated ingredients and techniques"
    }
}

In [ ]:
# Cell 6: Interactive Pipeline Execution

def list_available_pipelines():
    """Display available extraction pipelines with descriptions."""
    print("\nAvailable Extraction Pipelines:")
    print("-" * 50)
    for key, config in EXTRACTION_CONFIGS.items():
        print(f"\n{config['name']} (key: '{key}')")
        print(f"Description: {config['description']}")
        print(f"Output directory: {config['output_dir']}")
    print("\n" + "-" * 50)

def run_extraction_pipeline(extraction_type: str, input_dir: str):
    """
    Run the specified extraction pipeline.

    Args:
        extraction_type (str): Type of extraction to perform
        input_dir (str): Directory containing input PDFs
    """
    if extraction_type not in EXTRACTION_CONFIGS:
        print(f"Error: Invalid extraction type '{extraction_type}'")
        list_available_pipelines()
        return

    config = EXTRACTION_CONFIGS[extraction_type]
    print(f"\nRunning: {config['name']}")
    print(f"Input directory: {input_dir}")
    print(f"Output directory: {config['output_dir']}")

    try:
        # Initialize and run pipeline
        pipeline = MenuExtractionPipeline(config['output_dir'])
        pipeline.process_directory(input_dir, config['prompt'])
        print(f"\nExtraction completed successfully!")
        print(f"Results saved in: {config['output_dir']}")

    except Exception as e:
        print(f"\nError during extraction: {str(e)}")

In [ ]:
# Example usage:
list_available_pipelines()

In [ ]:
raw_data_dir = "/content/Raw_data_pdfs"

# To run a specific pipeline, uncomment and modify one of these lines:
run_extraction_pipeline("metadata", raw_data_dir)
run_extraction_pipeline("menu_basic", raw_data_dir)
run_extraction_pipeline("menu_constrained", raw_data_dir)